In [1]:
#
import numpy as np 
import pandas as pd 
import joblib


from ensemble import MajorityVotingEnsemble 
label_encoder = joblib.load("label_encode_main.joblib")
vectorizer = joblib.load("tfidf_vectorizer_main.joblib")

In [2]:
#
def patch_ensemble(ensemble):
    if not hasattr(ensemble, "weights"):
        ensemble.weights = [1.0] * len(ensemble.models)
    if not hasattr(ensemble, "tie_breaker_order"):
        ensemble.tie_breaker_order = list(range(len(ensemble.models)))
    if not hasattr(ensemble, "n_classes"):
        ensemble.n_classes = None
    return ensemble

In [3]:
#
model = joblib.load("majority_ensemble.pkl")

In [4]:
#
import pytesseract
import cv2
import pubchempy as pcp
from PIL import Image
import matplotlib.pyplot as plt

In [5]:
#barcode libraies 
#
import ctypes
import cv2
from pyzbar import pyzbar
from pyzbar import zbar_library

In [6]:
dll_path = r"C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\pyzbar\libzbar-64.dll"
zbar_library.load = lambda: ctypes.cdll.LoadLibrary(dll_path)

image_path = "Screenshot 2025-06-08 161620.png" 
image = cv2.imread('Screenshot 2025-06-08 161620.png')


In [7]:
#img -> text and barcode
#image then convert it into 
image = cv2.imread('Screenshot 2025-06-08 161620.png')  
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# remove noise and improve clarity
gray = cv2.GaussianBlur(gray, (3, 3), 0)

In [8]:
#barcode detection and barcode number 
if image is None:
    print(f"❌ Could not load image: {image_path}")
else:
    barcodes = pyzbar.decode(gray)#image

    if not barcodes:
        print("⚠️ No barcodes found.")
    else:
        for barcode in barcodes:
            x, y, w, h = barcode.rect
            data = barcode.data.decode('utf-8')
            type_ = barcode.type

            print(f"✅ Detected {type_} barcode: {data}")

            # Draw rectangle and label
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, f"{data} ({type_})", (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # ✅ Step 5: Show image with annotations
        # cv2.imshow("Barcode Detection", image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

✅ Detected EAN13 barcode: 0858099003002


In [9]:
# def barcode_detection(gray):
#     barcodes = pyzbar.decode(gray)#image

#     if not barcodes:
#         print("⚠️ No barcodes found.")
#     else:
#         for barcode in barcodes:
#             x, y, w, h = barcode.rect
#             data = barcode.data.decode('utf-8')
#             type_ = barcode.type
#             print(f"✅ Detected {type_} barcode: {data}")
#             title, brand, category= get_general_product_info(data)
#             if len(title) and len(description) == 0:
#                 print("No Details found")
#                 break
#             else:
#                 print(title, brand, category)

    
# barcode_detection(gray)

In [10]:
#img-> text
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\dhruv\AppData\Local\Programs\Tesseract-OCR\tesseract.exe' #tesseract path
text = pytesseract.image_to_string(gray)
print("Extracted Ingredients:")
print(len(text))

Extracted Ingredients:
964


In [11]:
#preprocessing
import re

def preprocess_text(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip().lower()
    return text
preprocess_text(text)
print((text))


CALIFORNIA PROPOSITION 65
stote of Colifornio to couse birth defects or other reproductive horm

Tngr “ym Silas Hota. Nicotine, FCC Grade Vegetable Glycerin,

[etre Fos, lv Ac. Westin conten 6 8 per
contri

Made in China.
Flavors mode in USA
©2012 LOEC, in.

blu™ ond blu eCigs® are trademarks of Lorillord Technologies, Inc.

1 serving per pack
Serving size 1 pack (32.35)
cesar appara

Calories 150

Trans Fat 09
Cholesterol m9
Sodium 1059 ot

INGREDIENTS
MILK CHOCOLATE (SUGAR
CHOCOLATE MILK. COCOA
BL T T. SOY
LE

(WHEAT FLOUR. NIACIN

IRON, AMINE M

RIBOFLAVIN, FOLIC ACID)

MODIFIED SAGO STARCH. SAI

SOYBEAN OIL, CORN SYRUP.

BARLEY MALT EXTRACT

LEAVENING (BAKING SODA. YEAS

AND/OR AMMONIUM

BICARBONATE)]. LESS THAN 2

CORNSTARCH, CORN SYRUP.

DEXTRIN. SALT. COLORING

(INCLUDES BLUE 1 LAKE, RED 40.
LLOW 6, YELLOW S$. BLUE 1, RED

40 LAKE, YELLOW 6 LAK!

SLAKE

CARNAUBA WAX. GUM ACACIA




In [12]:
#Extract chemiclas from text using spacy
import spacy
nlp = spacy.load("en_ner_bc5cdr_md")
# text = "Sodium Lauryl Sulfate, Propylparaben, Aloe Vera"
doc = nlp(text)

chemicals_ = [ent.text for ent in doc.ents if ent.label_ == "CHEMICAL"]
if len(chemicals_) == 0 :
    print("No Chemicals are detected ")
else:
    chemicals = list(dict.fromkeys(chemicals_))
    print("Detected chemicals")
    print("Extracted chemicals:", chemicals)

C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\spacy\language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


Detected chemicals
Extracted chemicals: ['nicotine', 'Nicotine', 'FCC', 'Glycerin', 'Cholesterol', 'Sodium', 'S$.']


In [13]:
# prediction thorugh ml momdel
def predict_product_category(text):
    clean_text = preprocess_text(text) # preproceessing is getting done 
    X_vec = vectorizer.transform([clean_text])
    predicted_label = model.predict(X_vec)[0]
    return predicted_label
    
predicted_category = predict_product_category(text)
print("Predicted Category:", predicted_category)

Predicted Category: 2


In [14]:
text #do some preprocessing to remove stopwordsd, punctuation \n "" etc

'CALIFORNIA PROPOSITION 65\nWarning: This product contoins nicotine, o chemical known to the\nstote of Colifornio to couse birth defects or other reproductive horm\n\nTngr “ym Silas Hota. Nicotine, FCC Grade Vegetable Glycerin,\n\n[etre Fos, lv Ac. Westin conten 6 8 per\ncontri\n\nMade in China.\nFlavors mode in USA\n©2012 LOEC, in.\n\nblu™ ond blu eCigs® are trademarks of Lorillord Technologies, Inc.\n\n1 serving per pack\nServing size 1 pack (32.35)\ncesar appara\n\nCalories 150\n\nTrans Fat 09\nCholesterol m9\nSodium 1059 ot\n\nINGREDIENTS\nMILK CHOCOLATE (SUGAR\nCHOCOLATE MILK. COCOA\nBL T T. SOY\nLE\n\n(WHEAT FLOUR. NIACIN\n\nIRON, AMINE M\n\nRIBOFLAVIN, FOLIC ACID)\n\nMODIFIED SAGO STARCH. SAI\n\nSOYBEAN OIL, CORN SYRUP.\n\nBARLEY MALT EXTRACT\n\nLEAVENING (BAKING SODA. YEAS\n\nAND/OR AMMONIUM\n\nBICARBONATE)]. LESS THAN 2\n\nCORNSTARCH, CORN SYRUP.\n\nDEXTRIN. SALT. COLORING\n\n(INCLUDES BLUE 1 LAKE, RED 40.\nLLOW 6, YELLOW S$. BLUE 1, RED\n\n40 LAKE, YELLOW 6 LAK!\n\nSLAKE\n\nC

In [15]:
# #advance of predicted category
# def predict_category_with_fallback(text, ensemble_model, vectorizer, label_encoder, threshold=0.45):
#     clean_text = preprocess_text(text)
#     X_vec = vectorizer.transform([clean_text])

#     # Get soft-voted probabilities (prefer ensemble's predict_proba if available)
#     if hasattr(ensemble_model, "predict_proba"):
#         proba = ensemble_model.predict_proba(X_vec)[0]
#     else:
#         proba = get_proba(ensemble_model, X_vec)

#     max_confidence = float(np.max(proba))
#     label_idx = int(np.argmax(proba))

#     # Decode to original string label using label encoder
#     try:
#         predicted_label = label_encoder.inverse_transform([label_idx])[0]
#     except Exception:
#         # fallback if label encoder can't map: use numeric index
#         predicted_label = label_idx

#     if max_confidence < threshold:
#         return "others", proba
#     return predicted_label, proba


In [16]:
# category, proba = predict_category_with_fallback(text, model, vectorizer, label_encoder)
# print("Predicted Category:", category)

In [17]:
# print("Confidence Scores:", dict(zip(label_encoder.classes_, proba)))

In [18]:
#more advanced
def preprocess_text(text):
    return text.lower().replace(",", " ")

def predict_category_with_fallback(text, model, vectorizer, label_encoder, threshold=0.45):
    clean_text = preprocess_text(text)
    X_vec = vectorizer.transform([clean_text])
    
    proba = model.predict_proba(X_vec)[0]
    max_confidence = np.max(proba)
    predicted_index = np.argmax(proba)
    predicted_label = label_encoder.inverse_transform([predicted_index])[0]

    if max_confidence < threshold:
        return "others", max_confidence
    return predicted_label, max_confidence


category, confidence = predict_category_with_fallback(text, model, vectorizer, label_encoder)
print(f"Predicted Category: {category} | Confidence: {confidence:.2f}")


Predicted Category: food | Confidence: 0.67


In [19]:
def calculate_harm_score(chemicals, category, harm_scores_csv):
    harm_df = pd.read_csv(harm_scores_csv)
    
    # Column for the specific category
    category_col = f"harm_score_{category}"
    if category_col not in harm_df.columns:
        raise ValueError(f"Category '{category}' not found in harm score table.")

    scores = []
    for chem in chemicals:
        row = harm_df[harm_df['chemical_name'].str.lower() == chem.lower()]
        if not row.empty:
            score = row[category_col].values[0]
            scores.append(score)

    if not scores:
        return None, None  # No chemicals matched

    avg_score = sum(scores) / len(scores)

    # Risk Level
    if avg_score <= 30:
        risk = "Low Risk"
    elif avg_score <= 70:
        risk = "Moderate Risk"
    else:
        risk = "High Risk"

    return avg_score, risk


# chemicals_list = ["Sodium Benzoate", "Citric Acid"," Natural Flavours", "Xanthan Gum", "Sugar"]  # From OCR
# predicted_category = "cosmetic"

avg_score, risk_level = calculate_harm_score(chemicals, category, "chemical_harmness_category.csv")

print(f"Average Harm Score: {avg_score:.2f}%")
print(f"Risk Level: {risk_level}")
print(f"Category :{category}")

Average Harm Score: 36.40%
Risk Level: Moderate Risk
Category :food


In [20]:
import pandas as pd
harm_df=pd.read_csv("chemical_harmness_category.csv")
def calculate_harm_score(chemicals, category):
    # harm_df = pd.read_csv(harm_scores_csv)
    
    category_col = f"harm_score_{category.lower()}"
    if category_col not in harm_df.columns:
        raise ValueError(f"Category '{category}' not found in harm score table.")
    
    individual_scores = {}
    
    for chem in chemicals:
        row = harm_df[harm_df['chemical_name'].str.lower() == chem.lower()]
        if not row.empty:
            score = row[category_col].values[0]
            individual_scores[chem] = score

    if not individual_scores:
        return {}, None, None  # No matching chemicals

    avg_score = sum(individual_scores.values()) / len(individual_scores)

    if avg_score <= 30:
        risk = "Low Risk"
    elif avg_score <= 70:
        risk = "Moderate Risk"
    else:
        risk = "High Risk"

    return individual_scores, avg_score, risk
scores, avg_score, risk_level = calculate_harm_score(chemicals, category)
print(scores, avg_score, risk_level)
#also write each harm title through manually coding 

{'nicotine': 80, 'Nicotine': 80, 'Glycerin': 5, 'Cholesterol': 12, 'Sodium': 5} 36.4 Moderate Risk


In [21]:
import requests

def get_general_product_info(barcode):
    url = f"https://api.upcitemdb.com/prod/trial/lookup?upc={barcode}"
    
    response = requests.get(url)
    if response.status_code != 200:
        print("❌ Failed to fetch data from UPCitemdb API.")
        return None
    
    data = response.json()
    
    if not data.get('items'):
        print("❌ No product found for this barcode.")
        return None

    item = data['items'][0]
    title = item.get('title', 'N/A')
    brand = item.get('brand', 'N/A')
    category = item.get('category', 'N/A')
    description = item.get('description', 'N/A')

    print(f"✅ Product Title: {title}")
    print(f"Brand: {brand}")
    print(f"Category: {category}")
    print(f"Description: {description}")

    return {
        'title': title,
        'brand': brand,
        'category': category
    }

# Example usage:
# get_general_product_info("028400642511")
# get_general_product_info("012993441012")  # Sample barcode
get_general_product_info(data)

❌ No product found for this barcode.


In [22]:
# Mapping category keywords to harm score columns
category_mapping = {
    'food': 'harm_score_food',
    'cosmetic': 'harm_score_cosmetic',
    'personal care': 'harm_score_personal_care',
    'cleaning': 'harm_score_cleaning',
    'stationery': 'harm_score_stationery',
    'household': 'harm_score_household',
    'medicine': 'harm_score_medicine',
    'others': 'harm_score_others'
}

In [23]:
api_category = "Food, Beverages & Tobacco > Food Items > Cooking & Baking Ingredients > Flour"

# Find matching key in mapping
matched_key = next((key for key in category_mapping if key in api_category.lower()), 'others')


harm_score_column = category_mapping[matched_key]

print("Using column:", harm_score_column)


Using column: harm_score_food


In [24]:
import pandas as pd

# Your chemical list from the detected product (example)
# chemicals = ['Citric Acid', 'Xanthan Gum', 'Sugar']

# Your harm score dataframe
df = pd.read_csv("chemical_harmness_category.csv")

# Filter relevant chemicals
filtered_df = df[df['chemical_name'].isin(chemicals)]

# Sum the relevant harm score column
avg_score = filtered_df[harm_score_column].sum()/len(filtered_df)
if avg_score <= 30:
     risk = "Low Risk"
elif avg_score <= 70:
     risk = "Moderate Risk"
else:
     risk = "High Risk"

   


print(f"Total Harm Score ({matched_key}):", avg_score)
print(f"Risk Level : {risk}")


Total Harm Score (food): 36.4
Risk Level : Moderate Risk


In [25]:
chemicals


['nicotine', 'Nicotine', 'FCC', 'Glycerin', 'Cholesterol', 'Sodium', 'S$.']

In [26]:
#50 seconda without reducing with 26 tabs

In [27]:
#based on wieght theier harmness will be measured like sugar =10 low sugar 90m high risk

In [55]:
def function(img_path):
    image = cv2.imread(img_path)  
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    text = pytesseract.image_to_string(gray) #img to text 
    clean_text = preprocess_text(text)  #preprocessing
    

    
    #chemiclas extraction
    doc = nlp(clean_text)
    chemicals = [ent.text for ent in doc.ents if ent.label_ == "CHEMICAL"]
    chemicals = list(set(chemicals))

    if len(chemicals) == 0 :
        print("No Chemicals are detected ")
    else:
        # chemicals = list(dict.fromkeys(chemicals))
        print("Detected chemicals")
        print("Extracted chemicals:", chemicals)

    process_barcode_image(gray, category_mapping, chemicals)

    
    #model
    category, confidence = predict_category_with_fallback(clean_text, model, vectorizer, label_encoder, threshold=0.45)
    print(f"Predicted Category: {category} | Confidence: {confidence:.2f}")
 
    individual_scores, avg_score, risk = calculate_harm_score(chemicals, category)
    print(f"Average Harm Score: {avg_score:.2f}%")
    print(f"Risk Level: {risk}")
    print(f"Category :{category}")
    print(f"Individual :{individual_scores}")#also write thier type

import cv2
import requests
import pandas as pd
from pyzbar import pyzbar

def process_barcode_image(gray, category_mapping, chemicals):
    # Load the image
    # image = cv2.imread(image_path)
    if image is None:
        print(f"❌ Could not load image: {image}")
        return
        
    barcodes = pyzbar.decode(gray)

    if not barcodes:
        print("⚠️ No barcodes found.")
        return

    for barcode in barcodes:
        x, y, w, h = barcode.rect
        barcode_data = barcode.data.decode('utf-8')
        # barcode_data="012993441012"
        type_ = barcode.type
        print(f"✅ Detected {type_} barcode: {barcode_data}")

        url = f"https://api.upcitemdb.com/prod/trial/lookup?upc={barcode_data}"
        response = requests.get(url)

        if response.status_code != 200:
            print("❌ Failed to fetch data from UPCitemdb API.")
            return

        data = response.json()
        if not data.get('items'):
            print("❌ No product found for this barcode.")
            return

        item = data['items'][0]
        title = item.get('title', 'N/A')
        brand = item.get('brand', 'N/A')
        category = item.get('category', 'N/A')
        description = item.get('description', 'N/A')

        print(f"✅ Product Title: {title}")
        print(f"Brand: {brand}")
        print(f"Category: {category}")

        api_category = category
        matched_key = next((key for key in category_mapping if key in api_category.lower()), 'others')
        harm_score_column = category_mapping[matched_key]
        print("Using column:", harm_score_column)

    
        filtered_df = harm_df[harm_df['chemical_name'].isin(chemicals)]

        if filtered_df.empty:
            print("⚠️ No matching chemicals found.")
            return

        avg_score = filtered_df[harm_score_column].sum() / len(filtered_df)
        if avg_score <= 30:
            risk = "Low Risk"
        elif avg_score <= 70:
            risk = "Moderate Risk"
        else:
            risk = "High Risk"

        print(f"Total Harm Score ({matched_key}): {avg_score:.2f}")
        print(f"Risk Level : {risk}")


In [56]:
img_path="Screenshot 2025-06-08 161620.png"

In [57]:
function(img_path) #increase chemical harm score 

Detected chemicals
Extracted chemicals: ['iron', 'riboflavin', 'barley malt extract', 'nicotine', 'cholesterol', 'glycerin', 'sodium', 'ammonium', 'amine', 'bicarbonate', 'gum acacia', 'cocoa', 'folic acid', 'niacin']
✅ Detected EAN13 barcode: 0858099003002
❌ No product found for this barcode.
Predicted Category: food | Confidence: 0.67
Average Harm Score: 20.00%
Risk Level: Low Risk
Category :food
Individual :{'iron': 10, 'nicotine': 80, 'cholesterol': 12, 'glycerin': 5, 'sodium': 5, 'folic acid': 8}


In [58]:
process_barcode_image(gray, category_mapping, chemicals)

✅ Detected EAN13 barcode: 0858099003002
❌ No product found for this barcode.
